In [8]:
URL = "http://10.206.0.2:8080"

!trino $URL --execute "show catalogs"


"bigquery"
"mongo"
"mysql"
"postgres"
"system"
"tpch"


In [9]:
#Q1

In [77]:
#Our database faker didn't generate any country codes in the US so I had to use the MV country code which the TA said is okay
!trino $URL --execute "\
SELECT s.cust_id, s.email, COUNT(r.res_id) AS reservation_count\
FROM postgres.final_project.reservation r \
JOIN mysql.final_project.shopper s ON r.cust_id = s.cust_id \
JOIN bigquery.final_project.currency c ON c.cntry_code=r.cnt_code\
WHERE r.city <> s.city \
AND c.curr_code = 'ANG' \
GROUP BY s.cust_id, s.email \
ORDER BY s.cust_id \
LIMIT 5"

"100ff8b0-30c4-42c6-857e-8a4f75cef62a","lindsey10@example.net","1"


In [14]:
#Q2

In [98]:
#Our database faker didn't produce any airlines with american airlines so I had to use another airline that produced the most results and the TA said it is okay
!trino $URL --execute "\
    SELECT t.cust_id, s.email, t.dep_airport, t.dep_date\
    FROM mongo.final_project.ticketing t\
    JOIN mysql.final_project.shopper s ON t.cust_id = s.cust_id\
    WHERE t.airline = 'Quinn, Sullivan and Garrison'\
        AND t.stops = 0\
        AND t.dep_date BETWEEN '2020-01-01T15:30:00.000000' AND '2024-12-31T15:30:00.000000'\
    ORDER BY t.dep_date\
    LIMIT 5"

"0edcab56-9dd1-4872-ab49-995d2b9a2bd0","anitabriggs@example.com","Allenhaven","2023-12-25T20:01:05.450196"


In [16]:
#Q3

In [97]:
!trino $URL --execute "\
    SELECT r.cust_id, s.email, SUM(r.pmt_amt) AS pmt_amt, SUM(t.tik_amt) AS tik_amt, SUM(r.pmt_amt) + SUM(t.tik_amt) AS trip_amount\
    FROM postgres.final_project.reservation r\
    JOIN mongo.final_project.ticketing t ON r.cust_id = t.cust_id\
    JOIN mysql.final_project.shopper s ON s.cust_id = r.cust_id\
    WHERE CAST(r.arr_date AS VARCHAR)= SUBSTRING(t.dep_date, 1, 10) AND CAST(r.dep_date AS VARCHAR) = SUBSTRING(t.arr_date, 1, 10)\
    GROUP BY r.cust_id, s.email\
    ORDER BY trip_amount DESC\
    LIMIT 5"

"0edcab56-9dd1-4872-ab49-995d2b9a2bd0","anitabriggs@example.com","8926.25","14388.0","23314.25"
"bda866f7-28d8-443e-96af-40ef2d49a6c0","jessicacollins@example.org","6304.98","9724.0","16028.98"
"d2ce4a8e-9666-477c-b72b-b23176ab2679","enguyen@example.org","4937.92","8226.0","13163.92"
"ce7df055-f3e7-4f59-a5ec-e94b4fbd7406","mcook@example.net","6305.49","5700.0","12005.49"
"b5ce43a4-75b1-497d-8608-792dbd3a5c94","karen09@example.org","6517.56","3840.0","10357.560000000001"


In [ ]:
#Trino Evaluation

In [104]:
!trino $URL --execute "\
CREATE TABLE mongo.final_project.new_table (\
    id INT,\
    name VARCHAR\
);"
# Create a new table in the MongoDB database:

CREATE TABLE


In [106]:
!trino $URL --execute "\
INSERT INTO mongo.final_project.new_table VALUES\
    (1, 'John'),\
    (2, 'Alice'),\
    (3, 'Bob');"
# Insert some sample data into the newly created table:

INSERT: 3 rows


In [108]:
!trino $URL --execute "\
ALTER TABLE mongo.final_project.new_table\
ADD COLUMN age INT;"
# Alter the structure of the new table in MongoDB:

ADD COLUMN


In [109]:
!trino $URL --execute "\
UPDATE mongo.final_project.new_table \
SET age = CASE\
    WHEN id = 1 THEN 25\
    WHEN id = 2 THEN 30\
    WHEN id = 3 THEN 28\
END;"
# Update the age values in the new table:

Query 20231207_054710_00095_hncrz failed: This connector does not support modifying table rows



In [107]:
!trino $URL --execute "\
SELECT n.id, n.name, s.email\
FROM mongo.final_project.new_table n\
JOIN mysql.final_project.shopper s ON n.id = s.cust_id\
LIMIT 5;"
# Perform a SELECT query involving a join between the new table and the MySQL shopper table:

Query 20231207_054439_00093_hncrz failed: line 1:110: Cannot apply operator: integer = varchar(200)
SELECT n.id, n.name, s.email FROM mongo.final_project.new_table n JOIN mysql.final_project.shopper s ON n.id = s.cust_id LIMIT 5



In [111]:
!trino $URL --execute "\
SELECT s.cust_id, s.email, COUNT(r.res_id) AS reservation_count\
FROM postgres.final_project.reservation r \
JOIN mysql.final_project.shopper s ON r.cust_id = s.cust_id \
WHERE r.city <> s.city \
AND r.cnt_code = 'MV'\
GROUP BY s.cust_id, s.email \
ORDER BY s.cust_id \
LIMIT 5"
#Finds shoppers who have reservations in a different city from their own city and and are in the country code MV For those shoppers it returns their cust_id, email, along with their reservation count and orders the results by cust_id and limits the results to 5 records.


"872344fd-60af-4183-a03c-d56c8f8b0805","tiffanyblackwell@example.net","1"
"97e33f79-faf1-45d5-8a0f-1d8bc53f3022","eserrano@example.net","1"
"bda866f7-28d8-443e-96af-40ef2d49a6c0","jessicacollins@example.org","1"


In [110]:
!trino $URL --execute "\
    SELECT t.cust_id, s.email, t.dep_airport, t.dep_date\
    FROM mongo.final_project.ticketing t\
    JOIN mysql.final_project.shopper s ON t.cust_id = s.cust_id\
    WHERE t.airline = 'Quinn, Sullivan and Garrison'\
        AND t.stops = 1\
        AND t.dep_date BETWEEN '2021-01-01T15:30:00.000000' AND '2023-12-31T15:30:00.000000'\
    ORDER BY t.dep_date\
    LIMIT 5"
#Finds the shoppers who bought tickets on Quinn, Sullivan and Garrison with a departure date between 01/01/2021 and 12/31/2023 and returns the shopper’s cust_id, email,dep_airport and dep_date. It then orders the results by dep_date and Limits the results to 5 records

In [113]:
!trino $URL --execute "\
DROP TABLE mongo.final_project.new_table;"
# Drop the new table in MongoDB:

DROP TABLE
